# Внешние отраслевые источники

In [1]:

import re
import time
import pickle
from tqdm import tqdm

import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup
import html2text
from datetime import datetime
from pprint import pprint


def get_soup(link):
    HEADERS = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
    }
    soup = None
    response = requests.get(link, headers=HEADERS)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
    else:
        print(f"Ошибка запроса: {response.status_code}")
    return soup


def convert_to_text(soup):
    html_converter = html2text.HTML2Text()
    html_converter.ignore_links = True
    html_converter.body_width = 0
    markdown_text = html_converter.handle(str(soup))
    return markdown_text


def convert_date(date_str):
    months = {
        'январь': 1, 'февраль': 2, 'март': 3, 'апрель': 4,
        'май': 5, 'июнь': 6, 'июль': 7, 'август': 8,
        'сентябрь': 9, 'октябрь': 10, 'ноябрь': 11, 'декабрь': 12
    }
    month_str, year_str = date_str.lower().split()
    month = months.get(month_str)
    year = int(year_str)

    return datetime(year, month, 1)

Результатом пасинга с каждого сайта будет аля jsonlines формата:

## Формат данных 

Результат представляет собой массив словарей, где каждый словарь содержит информацию об одном аналитическом материале. Каждый словарь имеет следующие ключи:

### Ключи словаря:
1. **`link`** (`str`):
   - Ссылка на страницу с аналитическим материалом. Может быть как полная, так и относительная
   - Пример: `"/news/12345"`.

2. **`date`** (`str`):
   - Дата публикации материала.
   - Пример: `"2023-10-15"`.

3. **`name`** (`str`):
   - Название материала на русском языке.
   - Пример: `"Анализ рынка ИТ в 2023 году"`.

4. **`name_en`** (`str`):
   - Название материала на английском языке, извлеченное из ссылки.
   - Пример: `"it-market-analysis-2023"`.

5. **`tags`** (`list` of `str`):
   - Список тегов, связанных с материалом.
   - Пример: `["ИТ", "аналитика", "2023"]`.

6. **`pdf_links`** (`list` of `str`):
   - Список ссылок на PDF-файлы, связанные с материалом.
   - Пример: `["https://b1.ru/local/assets/surveys/it-market-analysis-2023.pdf"]`.

7. **`text`** (`str`):
   - Основной текст материала из статьи на сайте (в pdf больше информации), очищенный от лишних элементов (контакты, реклама, скрипты и т.д.).
   - Пример: `"В 2023 году рынок ИТ показал рост на 14%..."`.

8. **`pdfs`** (`list` of `dict`):
   - Список словарей, содержащих информацию о скачанных PDF-файлах. Каждый словарь содержит:
     - **`name`** (`str`): Название PDF-файла.
       - Пример: `"it-market-analysis-2023.pdf"`.
     - **`content`** (`bytes`): Бинарное содержимое PDF-файла.

---

### Пример элемента списка `analytics`:
```json
{
    "link": "/news/12345",
    "date": "2023-10-15",
    "name": "Анализ рынка ИТ в 2023 году",
    "name_en": "it-market-analysis-2023",
    "tags": ["ИТ", "аналитика", "2023"],
    "pdf_links": [
        "https://b1.ru/local/assets/surveys/it-market-analysis-2023.pdf"
    ],
    "text": "В 2023 году рынок ИТ показал рост на 14%...",
    "pdfs": [
        {
            "name": "it-market-analysis-2023.pdf",
            "content": "binary_pdf_content_here"
        }
    ]
}
```

## b1-surveys
Парсим данные со страницы https://b1.ru/b1-surveys

In [2]:
URL = 'https://b1.ru/'
SURVEYS_URL = URL + 'b1-surveys/'

session = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
session.mount('http://', HTTPAdapter(max_retries=retries))
session.mount('https://', HTTPAdapter(max_retries=retries))

soup = get_soup(SURVEYS_URL)
pagination = soup.find('div', class_='pagination')
page_links = [a['href'] for a in pagination.find_all('a') if '?PAGEN_2=' in a['href']]
last_page = max([int(a.split('?PAGEN_2=')[-1]) for a in page_links])
analytics = []
for i in tqdm(range(1, last_page + 1)):
    soup = get_soup(SURVEYS_URL + '?PAGEN_2=' + str(i))
    time.sleep(1)
    for block in soup.find_all('div', class_=re.compile(r"^news-block__gridItem triangleWrap--hover")):
        if block:
            link = block['src-href']
            if not 'services' in link:
                analytics_block = {}
                analytics_block['link'] = link
                analytics_block['date'] = datetime.strptime(block.find('p', class_='news-block__date').text, '%d.%m.%Y')
                analytics_block['name'] = block.find('h2', class_='news-block__title').text
                analytics_block['name_en'] = link.strip().strip('/').split('/')[-1]
                
                time.sleep(1)
                analytics_soup = get_soup(URL + link)
                tags_block = analytics_soup.find('div', class_=re.compile(r"tezis-list-block__tags"))
                if tags_block:
                    analytics_block['tags'] = [a.text for a in tags_block.find_all('a', class_=re.compile(r"tag"))]
                else:
                    analytics_block['tags'] = []
                
                analytics_block['pdf_links'] = [f'{URL}{a['href']}' for a in analytics_soup.find_all('a', href=True) if a['href'].endswith('.pdf')]
                analytics_block['pdf_links'].append(f'{URL}local/assets/surveys/{analytics_block['name_en']}.pdf')

                main_text = analytics_soup.find('main')
                for el in main_text.find_all('section', attrs={"data-block-name": "Контакты"}):
                    el.extract()
                for el in main_text.find_all('section', attrs={"data-block-name": "СКАЧАТЬ ПОЛНУЮ ВЕРСИЮ"}):
                    el.extract()
                for el in main_text.find_all('div', attrs={"class": "page-cover"}):
                    el.extract() 
                for el in main_text.find_all('script'):
                    el.extract()
                for el in main_text.find_all('style'):
                    el.extract()
                if tags_block:
                    for el in tags_block:
                        el.extract()
                analytics_block['text'] = convert_to_text(main_text)
                
                pdfs_content = []
                pdfs_names = []
                for url in analytics_block['pdf_links']:
                    for i in range(10):
                        try:
                            response = session.get(url, stream=True)
                            if response.status_code == 200:
                                pdf_name = url.strip().strip('/').split('/')[-1]
                                pdf_content = response.content
                                if pdf_name not in pdfs_names:
                                    pdfs_names.append(pdf_name)
                                    pdfs_content.append({"name": pdf_name, "content": pdf_content})
                                break
                        except:
                            time.sleep(3)
                        
                analytics_block['pdfs'] = pdfs_content
                analytics.append(analytics_block)

with open('b1_analytics.pkl', 'wb') as f:
    pickle.dump(analytics, f)

100%|██████████| 7/7 [50:51<00:00, 435.90s/it]   


## kamaflow
Парсим данные с сайта https://kamaflow.com/ru/blog/

In [ ]:
URL = 'https://strategy.ru'

session = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
session.mount('http://', HTTPAdapter(max_retries=retries))
session.mount('https://', HTTPAdapter(max_retries=retries))

url = URL + "/api/v1/research/research"
params = {"limit": 10, "offset": 0}
response = session.get(url, params=params)

if response.status_code == 200:
    data = response.json()
else:
    print(f"Ошибка: {response.status_code}")
    print(response.text)

researches_links = []
k = 10
for i in range(0, data['count'] + 1, k):
    params = {"limit": k, "offset": i}
    response = session.get(url, params=params)
    data = response.json()
    researches_links += data['researches']

In [7]:
researches = []
for research_block in tqdm(researches_links):
    link = URL + research_block['absolute_url']
    date = convert_date(research_block['published_at'])
    tags = [ind['title'] for ind in research_block['industries']]
    name = research_block['title']
    text_preview = research_block['preview']
    
    info = {'link': link, 'date': date, 'name': name, 'text_preview': text_preview, 'tags': tags}
    time.sleep(1)
    for i in range(10):
        try:
            soup = get_soup(info['link'])
            publication_div = soup.find('div', class_='publication__text tiny')
            pdf_links = []
            for btn in publication_div.find_all('a', class_='btn-download'):
                pdf_links.append(URL + btn.extract()['href'].replace('../', '/'))
            for el in publication_div.find_all('script'):
                    el.extract()
            for el in publication_div.find_all('style'):
                el.extract()
            info['text'] = convert_to_text(publication_div)
            info['pdf_links'] = list(set(pdf_links))
            pdfs_content = []
            for pdf_link in info['pdf_links']:
                response = session.get(pdf_link, stream=True)
                if response.status_code == 200:
                    pdf_name = url.strip().strip('/').split('/')[-1]
                    pdf_content = response.content
                    pdfs_content.append({"name": info['name'], "content": pdf_content})
            info['pdfs'] = pdfs_content
            break
        except:
            time.sleep(5)
    
    researches.append(info)
with open('kamaflow_researches.pkl', 'wb') as f:
    pickle.dump(researches, f)

100%|██████████| 39/39 [11:20<00:00, 17.45s/it]


# 